## Mortgage

In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL,lang='english',check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.27.6

#### No Stress

In [2]:
m = ["Mortgage"
        ,{"originBalance": 12000.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 30
            ,"freq": "Monthly"
            ,"type": "Level"
            ,"originDate": "2021-02-01"}
        ,{"currentBalance": 10000.0
            ,"currentRate": 0.075
            ,"remainTerm": 20
            ,"status": "Current"}]

### ("Mortgage")

In [3]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", None, None, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

r[0].head(5)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-12-01,10000.00,0.00,0.00,0,0,0,0,0.075,None,None,0.00,0,0,0,0,0
2022-01-01,9529.05,470.95,62.50,0,0,0,0,0.075,None,None,470.95,0,0,0,0,0
2022-02-01,9055.14,473.91,59.55,0,0,0,0,0.075,None,None,944.86,0,0,0,0,0
2022-03-01,8578.28,476.86,56.59,0,0,0,0,0.075,None,None,1421.72,0,0,0,0,0
2022-04-01,8098.43,479.85,53.61,0,0,0,0,0.075,None,None,1901.57,0,0,0,0,0


#### Default Assumption

`{"CDR": x}` -> using constant annualized default rate

In [4]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", {"CDR":0.04}, None, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

r[0].head(5)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-12-01,10000.00,0.00,0.00,0,0.00,0,0.00,0.075,None,None,0.00,0,0,0.00,0,0.00
2022-01-01,9496.06,469.33,62.28,0,34.61,0,34.61,0.075,None,None,469.33,0,0,34.61,0,34.61
2022-02-01,8992.57,470.63,59.14,0,32.86,0,32.86,0.075,None,None,939.96,0,0,67.47,0,67.47
2022-03-01,8492.37,472.09,56.02,0,28.11,0,28.11,0.075,None,None,1412.05,0,0,95.58,0,95.58
2022-04-01,7989.58,473.40,52.89,0,29.39,0,29.39,0.075,None,None,1885.45,0,0,124.97,0,124.97


`{"CDR":[x1,x2,x3]}` -> using a curve, starting from date which determined from `remainTerms`

In [5]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", {"CDR":[0.01,0.02,0.03,0.0]}, None, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

r[0].head(5)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-12-01,10000.00,0.00,0.00,0,0.00,0,0.00,0.075,None,None,0.00,0,0,0.00,0,0.00
2022-01-01,9520.91,470.56,62.44,0,8.53,0,8.53,0.075,None,None,470.56,0,0,8.53,0,8.53
2022-02-01,9031.90,472.69,59.40,0,16.32,0,16.32,0.075,None,None,943.25,0,0,24.85,0,24.85
2022-03-01,8536.30,474.53,56.31,0,21.07,0,21.07,0.075,None,None,1417.78,0,0,45.92,0,45.92
2022-04-01,8058.81,477.49,53.35,0,0.00,0,0.00,0.075,None,None,1895.27,0,0,45.92,0,45.92


`{"DefaultByAmt":("totalDefaultAmt",[x1,x2,x3...])`: allocate default balance in cashflow projection

In [6]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", {"ByAmount":(100,[0.5,0.25,0.25])}, None, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

r[0].head(5)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-12-01,10000.00,0.00,0.00,0,0,0,0,0.075,None,None,0.00,0,0,0,0,0
2022-01-01,9481.39,468.61,62.18,0,50,0,50,0.075,None,None,468.61,0,0,50,0,50
2022-02-01,8986.10,470.29,59.10,0,25,0,25,0.075,None,None,938.90,0,0,75,0,75
2022-03-01,8489.19,471.91,56.00,0,25,0,25,0.075,None,None,1410.81,0,0,100,0,100
2022-04-01,8014.33,474.86,53.05,0,0,0,0,0.075,None,None,1885.67,0,0,100,0,100


#### Prepayment Assumption

In [7]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", None, {"CPR":0.01}, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

r[0].head(5)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-12-01,10000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2022-01-01,9520.91,470.56,62.44,8.53,0,0,0,0.075,None,None,470.56,8.53,0,0,0,0
2022-02-01,9039.70,473.09,59.45,8.12,0,0,0,0.075,None,None,943.65,16.65,0,0,0,0
2022-03-01,8557.05,475.69,56.45,6.96,0,0,0,0.075,None,None,1419.34,23.61,0,0,0,0
2022-04-01,8071.50,478.25,53.43,7.30,0,0,0,0.075,None,None,1897.59,30.91,0,0,0,0


In [8]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", None, {"CPR":[0.01,0,0.02,0.03]}, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

r[0].head(5)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2021-12-01,10000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2022-01-01,9520.91,470.56,62.44,8.53,0,0,0,0.075,None,None,470.56,8.53,0,0,0,0
2022-02-01,9047.41,473.50,59.50,0.00,0,0,0,0.075,None,None,944.06,8.53,0,0,0,0
2022-03-01,8557.68,475.72,56.45,14.01,0,0,0,0.075,None,None,1419.78,22.54,0,0,0,0
2022-04-01,8058.12,477.46,53.34,22.10,0,0,0,0.075,None,None,1897.24,44.64,0,0,0,0


#### Recovery/Loss Assumption

Recovery by Lag

In [9]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", {"CDR":0.01}, None, {"Rate":0.3,"Lag":2}, None)
                                    ,None
                                    ,None)
                     ,read=True)

r[0].tail(10)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2023-01-01,3603.11,502.02,25.65,0,3.50,1.30,3.05,0.075,None,None,6319.87,0,0,77.02,20.85,48.77
2023-02-01,3095.32,504.72,22.50,0,3.07,1.14,2.66,0.075,None,None,6824.59,0,0,80.09,21.99,51.43
2023-03-01,2585.45,507.49,19.33,0,2.38,1.05,2.45,0.075,None,None,7332.08,0,0,82.47,23.04,53.88
2023-04-01,2073.02,510.23,16.14,0,2.20,0.92,2.14,0.075,None,None,7842.31,0,0,84.67,23.96,56.02
2023-05-01,1558.31,513.00,12.94,0,1.71,0.71,1.66,0.075,None,None,8355.31,0,0,86.38,24.67,57.68
2023-06-01,1041.23,515.76,9.73,0,1.32,0.66,1.54,0.075,None,None,8871.07,0,0,87.70,25.33,59.22
2023-07-01,521.81,518.57,6.50,0,0.85,0.51,1.19,0.075,None,None,9389.64,0,0,88.55,25.84,60.41
2023-08-01,0.00,521.37,3.25,0,0.44,0.39,0.92,0.075,None,None,9911.01,0,0,88.99,26.23,61.33
2023-09-01,0.00,0.00,0.00,0,0.00,0.25,0.59,0.000,None,None,9911.01,0,0,88.99,26.48,61.92


Recovery Timing

In [13]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", {"CDR":0.01}
                                                , None
                                                , {"Rate":0.3,"Timing":[0.5,0.25,0.25]}
                                                , None)
                                    ,None
                                    ,None)
                     ,read=True)
r[0].tail(10)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2023-01-01,3603.11,502.02,25.65,0,3.50,1.12,3.05,0.075,None,None,6319.87,0,0,77.02,22.09,48.77
2023-02-01,3095.32,504.72,22.50,0,3.07,1.00,2.66,0.075,None,None,6824.59,0,0,80.09,23.09,51.43
2023-03-01,2585.45,507.49,19.33,0,2.38,0.84,2.45,0.075,None,None,7332.08,0,0,82.47,23.93,53.88
2023-04-01,2073.02,510.23,16.14,0,2.20,0.73,2.14,0.075,None,None,7842.31,0,0,84.67,24.66,56.02
2023-05-01,1558.31,513.00,12.94,0,1.71,0.58,1.66,0.075,None,None,8355.31,0,0,86.38,25.24,57.68
2023-06-01,1041.23,515.76,9.73,0,1.32,0.47,1.54,0.075,None,None,8871.07,0,0,87.70,25.71,59.22
2023-07-01,521.81,518.57,6.50,0,0.85,0.33,1.19,0.075,None,None,9389.64,0,0,88.55,26.04,60.41
2023-08-01,0.00,521.37,3.25,0,0.44,0.21,0.92,0.075,None,None,9911.01,0,0,88.99,26.25,61.33
2023-09-01,0.00,0.00,0.00,0,0.00,0.09,0.59,0.000,None,None,9911.01,0,0,88.99,26.34,61.92
